In [1]:
!pip install datasets

     -------------------------------------- 474.6/474.6 KB 4.3 MB/s eta 0:00:00
     --------------------------------------- 21.5/21.5 MB 16.0 MB/s eta 0:00:00
     ------------------------------------- 224.5/224.5 KB 14.3 MB/s eta 0:00:00
     ------------------------------------- 324.5/324.5 KB 20.9 MB/s eta 0:00:00


     ---------------------------------------- 61.8/61.8 KB ? eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import multiprocessing

from time import time 
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from datasets import load_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !gdown --id 1N0U-i4PxT_DmTru0r7T5po-9f9cwGgB8 -O RedditComments2.csv
# !gdown --id 1dlV7huXQfim8brL7LIBy-M2kFR63P2Oh -O Reddit_metadata2.csv
# !gdown --id 1vT1g4Crj6s6z2YNjSyPx5Fvz5KTpBYSB -O annottated_reddit.csv

### Preparing reddit subset from original reddit data 

In [3]:
foundations = ['care','harm','fairness','cheating','loyalty','betrayal','authority','subversion','sanctity','degradation','liberty', 'oppression', 
               'non-moral', 'equality', 'proportionality', 'purity']

In [4]:
dataset = load_dataset("USC-MOLA-Lab/MFRC")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 49.74it/s]


Dataset csv downloaded and prepared to C:/Users/shafz/.cache/huggingface/datasets/USC-MOLA-Lab___csv/USC-MOLA-Lab--MFRC-d880200ad192d2ed/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 39.92it/s]


In [5]:
df_annotated = pd.DataFrame(dataset["train"])

In [6]:
df_annotated.head(5)

,text,subreddit,bucket,annotator,annotation,confidence
0,That particular part of the debate is especial...,europe,French politics,annotator03,Non-Moral,Confident
1,That particular part of the debate is especial...,europe,French politics,annotator01,Purity,Confident
2,That particular part of the debate is especial...,europe,French politics,annotator02,Thin Morality,Confident
3,"/r/france is pretty lively, with it's own ling...",europe,French politics,annotator03,Non-Moral,Confident
4,"/r/france is pretty lively, with it's own ling...",europe,French politics,annotator00,Non-Moral,Somewhat Confident


In [7]:
df_annotated['confidence'].value_counts()

Confident             44737
Somewhat Confident     8920
Not Confident          7527
Name: confidence, dtype: int64

In [8]:
df_annotated.to_csv('reddit_raw.csv')

In [9]:
reddit = df_annotated.loc[df_annotated['confidence']== 'Confident']
reddit['annotation'].value_counts()

Non-Moral                                          24186
Thin Morality                                       7321
Care                                                2560
Authority                                           1699
Equality                                            1686
                                                   ...  
Care,Thin Morality,Proportionality                     1
Purity,Proportionality,Equality                        1
Equality,Proportionality,Care,Loyalty,Authority        1
Purity,Care,Loyalty                                    1
Proportionality,Authority,Loyalty,Equality             1
Name: annotation, Length: 124, dtype: int64

In [10]:
idx = []
annos = {}
for i in range(len(reddit)):
  foundation = reddit["annotation"].iloc[i]
  if foundation.lower() in foundations:
    pass 
  else:
    annos[foundation] = 1
    idx.append(i)

idx_ = reddit.iloc[idx].index.values    
reddit = reddit.drop(idx_, axis = 0)
reddit = reddit.drop(['annotator', 'confidence'], axis = 1)
reddit["annotation"].value_counts()

Non-Moral          24186
Care                2560
Authority           1699
Equality            1686
Proportionality     1389
Loyalty              700
Purity               544
Name: annotation, dtype: int64

In [11]:
len(annos)

117

In [12]:
reddit_resampled = reddit.drop(reddit[reddit['annotation'] == 'Non-Moral'].sample(frac=0.93).index)
reddit_resampled = reddit_resampled.drop(reddit_resampled[reddit_resampled['annotation'] == 'Care'].sample(frac=0.40).index)

In [13]:
reddit_resampled["annotation"].value_counts()

Authority          1699
Non-Moral          1693
Equality           1686
Care               1536
Proportionality    1389
Loyalty             700
Purity              544
Name: annotation, dtype: int64

In [14]:
reddit_resampled.shape

(9247, 4)

In [15]:
reddit_resampled.to_csv('reddit.csv')